In [1]:
# import tabulate
import json
import re
import os
import pandas as pd



In [6]:
## p1 = r"(?P<misson_name>.+)_(?P<gamma>\d+(\.\d+)?)_(?P<delta>.+)_(?P<threshold>.+)_z"
## 
## file = "alpacafarm_0.75_5.0_6.0_z.jsonl"
## matcher1 = re.match(p1, file)
## misson_name = matcher1.group("misson_name")
## threshold = matcher1.group("threshold")
## 
## print(misson_name, threshold)

# g0.5 d5.0
# g0.25 d5.0
# g0.75 d5.0
# g0.5 d2.0
# g0.25 d2.0
# g0.9 d5.0

In [8]:
# load data from pred


df = pd.DataFrame(columns=["model_name", "mission_category", "mode", "gamma", "delta", "threshold", "bl_type", "z_score", "sum"])

input_dir = "./pred"
p = r"(?P<model_name>.+)_(?P<mode>old|v2|gpt|new|no)_g(?P<gamma>.+)_d(?P<delta>\d+(\.\d+)?)"
p1 = r"(?P<misson_name>[a-zA-Z_]+)_(?P<gamma>\d+(\.\d+)?)_(?P<delta>.+)_z"

num = 0
# get all files from input_dir
for subfolder in os.listdir(input_dir):
    print("subfolder is:", subfolder)
    matcher = re.match(p, subfolder)
    model_name = matcher.group("model_name")
    mode = matcher.group("mode")
    gamma = matcher.group("gamma")
    delta = matcher.group("delta")
    
    bl_type = "None"
    bl_type = (subfolder.split("_")[-1]).split(".")[0]
    
    print(bl_type)
    if bl_type != "hard":
        if "old" in subfolder:
            bl_type = "soft"
        else:
            bl_type = "None"
        
        
    print(model_name, mode, gamma, delta, bl_type)
    
    z_score_path = os.path.join(input_dir, subfolder, "z_score")
    if os.path.exists(z_score_path):
        # print("subfolder is:", subfolder)
        files = os.listdir(z_score_path)
        
        for file in files:
            # print(file)
            # read jsons
            matcher1 = re.match(p1, file)
            if matcher1:
                misson_name = matcher1.group("misson_name")
                threshold = 6.0
            else:
                threshold = file.split("_")[-2]
            
            with open(os.path.join(z_score_path, file), "r") as f:
                data = json.load(f)

            # get data
            avarage_z = data["avarage_z"]
            sum = len(data["z_score_list"])
            # wm_pred_avarage = data["wm_pred_avarage"]
            
            temp = pd.DataFrame({
                "model_name": [model_name],
                "mode": [mode],
                "mission_category": [misson_name],
                "gamma": [gamma],
                "delta":[delta],
                "threshold": [threshold],
                "bl_type": [bl_type],
                "z_score": [avarage_z],
                "sum": [sum]})
            
            df = pd.concat([df, temp], ignore_index=True)
            num += 1

df.to_csv("z_score.csv")           
print(df)
print(num)
                



subfolder is: llama2-7b-chat-4k_gpt_g0.25_d10.0
d10
llama2-7b-chat-4k gpt 0.25 10.0 None
subfolder is: llama2-7b-chat-4k_gpt_g0.1_d10.0
d10
llama2-7b-chat-4k gpt 0.1 10.0 None
subfolder is: llama2-7b-chat-4k_old_g0.25_d15.0_hard
hard
llama2-7b-chat-4k old 0.25 15.0 hard
subfolder is: llama2-7b-chat-4k_old_g0.75_d5.0_hard
hard
llama2-7b-chat-4k old 0.75 5.0 hard
subfolder is: llama2-7b-chat-4k_v2_g0.25_d2.0
d2
llama2-7b-chat-4k v2 0.25 2.0 None
subfolder is: llama2-7b-chat-4k_old_g0.5_d2.0_hard
hard
llama2-7b-chat-4k old 0.5 2.0 hard
subfolder is: llama2-7b-chat-4k_old_g0.25_d5.0_hard
hard
llama2-7b-chat-4k old 0.25 5.0 hard
subfolder is: llama2-7b-chat-4k_gpt_g0.5_d5.0
d5
llama2-7b-chat-4k gpt 0.5 5.0 None
subfolder is: llama2-7b-chat-4k_old_g0.1_d5.0
d5
llama2-7b-chat-4k old 0.1 5.0 soft
subfolder is: llama2-7b-chat-4k_v2_g0.25_d10.0
d10
llama2-7b-chat-4k v2 0.25 10.0 None
subfolder is: llama2-7b-chat-4k_v2_g0.1_d5.0
d5
llama2-7b-chat-4k v2 0.1 5.0 None
subfolder is: llama2-7b-chat-4k

In [3]:
df_sorted = df.sort_values(by="z_score", ascending=True)
df_sorted.to_csv("z_score_sorted.csv")           
print(df_sorted)

            model_name             mission_name mode gamma delta  threshold  \
255  llama2-7b-chat-4k                      lcc   no   0.5   5.0        6.0   
238  llama2-7b-chat-4k                 hotpotqa  old   0.9   5.0        6.0   
242  llama2-7b-chat-4k  konwledge_understanding  old   0.9   5.0        6.0   
218  llama2-7b-chat-4k  konwledge_understanding   v2   0.5   2.0        6.0   
161  llama2-7b-chat-4k  konwledge_understanding  old   0.9   5.0        6.0   
..                 ...                      ...  ...   ...   ...        ...   
360  llama2-7b-chat-4k               multi_news  old   0.1  10.0        6.0   
12   llama2-7b-chat-4k                    qmsum  gpt   0.1  10.0        6.0   
276  llama2-7b-chat-4k               multi_news  old   0.1   5.0        6.0   
315  llama2-7b-chat-4k               multi_news  old   0.1  10.0        6.0   
15   llama2-7b-chat-4k               multi_news  gpt   0.1  10.0        6.0   

    bl_type    z_score  sum  
255    None  -0.91561

In [12]:
### process_mutual
df = pd.DataFrame(columns=["model_name", "mission_name", "ref_mode", "det_mode", "threshold", "z_score", "sum"])

input_dir = "./mutual_detect/llama2-7b-chat-4k"

num = 0
p1 = r"(?P<misson_name>[a-zA-Z_]+)_(?P<threshold>\d+(\.\d+)?)_z"

# get all files from input_dir
for subfolder in os.listdir(input_dir):
    print("subfolder is:", subfolder)
    
    ref_mode = subfolder.split("ref_")[1]
        
    print(ref_mode)
    
    for subsubfolder in os.listdir(os.path.join(input_dir,subfolder)):
        
        det_mode = subsubfolder.split("_z")[0]
        print(det_mode)
    
        z_score_path = os.path.join(input_dir, subfolder, subsubfolder)
        if os.path.exists(z_score_path):
        # print("subfolder is:", subfolder)
            files = os.listdir(z_score_path)
        
            for file in files:
                # print(file)
                # read jsons
                matcher1 = re.match(p1, file)
                if matcher1:
                    misson_name = matcher1.group("misson_name")
                    threshold = matcher1.group("threshold")
                with open(os.path.join(z_score_path, file), "r") as f:
                    data = json.load(f)

                # get data
                avarage_z = data["avarage_z"]
                sum = len(data["z_score_list"])
                # wm_pred_avarage = data["wm_pred_avarage"]
                
                temp = pd.DataFrame({
                    "model_name": [model_name],
                    "ref_mode": [ref_mode],
                    "mission_name": [misson_name],
                    "det_mode":[det_mode],
                    "threshold": [threshold],
                    "z_score": [avarage_z],
                    "sum": [sum]})
                
                df = pd.concat([df, temp], ignore_index=True)
                num += 1

df.to_csv("mutual_z_score.csv")           
print(df)
print(num)


subfolder is: ref_old_soft_g0.1_d10.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_hard_g0.25_d5.0
old_soft_g0.25_d15.0
v2_g0.25_d15.0
subfolder is: ref_old_hard_g0.25_d5.0
old_hard_g0.25_d5.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_soft_g0.25_d15.0
v2_g0.25_d15.0
subfolder is: ref_old_soft_g0.25_d15.0
old_soft_g0.25_d15.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_hard_g0.25_d5.0
v2_g0.25_d15.0
subfolder is: ref_gpt_g0.1_d10.0
gpt_g0.1_d10.0
old_soft_g0.1_d10.0
old_hard_g0.25_d5.0
old_soft_g0.25_d15.0
v2_g0.25_d15.0
subfolder is: ref_v2_g0.25_d15.0
v2_g0.25_d15.0
old_soft_g0.1_d10.0
gpt_g0.1_d10.0
old_hard_g0.25_d5.0
old_soft_g0.25_d15.0
            model_name             mission_name             ref_mode  \
0    llama2-7b-chat-4k               multi_news  old_soft_g0.1_d10.0   
1    llama2-7b-chat-4k               alpacafarm  old_soft_g0.1_d10.0   
2    llama2-7b-chat-4k                      lcc  old_soft_g0.1_d10.0   
3    llama2-7b-chat-4k              longform_qa  old_soft_g0.1_d10.0   
4   

In [19]:
# 导入transformers库
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch
import numpy as np
import random
import json

def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed_all(seed)
    
seed_everything(42)
model2path = json.load(open("config/model2path.json", "r"))
dataset2maxlen = json.load(open("config/dataset2maxlen.json", "r"))
model_name = "vicuna-13b"
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# 加载chatglm2-6b-32k模型的tokenizer
tokenizer = AutoTokenizer.from_pretrained(model2path[model_name], trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(model2path[model_name], trust_remote_code=True,
                                                  output_scores=True, return_dict_in_generate=True, 
                                                  torch_dtype=torch.bfloat16).to(device)

# 获取tokenizer的词表大小
vocab_size = model.config.padded_vocab_size
vocab_size_tokenizer = len(tokenizer.get_vocab().values())

# 打印词表大小
print("The vocab size of chatglm2-6b-32k tokenizer is", vocab_size_tokenizer)
print("The vocab size of chatglm2-6b-32k is", vocab_size)


Loading checkpoint shards: 100%|██████████| 7/7 [00:09<00:00,  1.36s/it]


The vocab size of chatglm2-6b-32k tokenizer is 64787
The vocab size of chatglm2-6b-32k is 65024


In [16]:
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM
import torch
import numpy as np
import random
import json

from generate import Generator
from argparse import Namespace


def seed_everything(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    torch.cuda.manual_seed_all(seed)
    
seed_everything(42)
model_name = "internlm-7b-8k"
model2path = json.load(open("config/model2path.json", "r"))
dataset2maxlen = json.load(open("config/dataset2maxlen.json", "r"))
dataset = "alpacafarm"
max_gen = dataset2maxlen[dataset]
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
tokenizer = AutoTokenizer.from_pretrained(model2path[model_name], trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
            model2path[model_name], device_map='auto', return_dict_in_generate=True, output_scores=True,trust_remote_code=True
        )

watermark_args = Namespace(mode="v2", gamma=0.25, delta=10.0, initial_seed=1234, dynamic_seed=None, bl_type="soft", num_beams=1, sampling_temp=0.7, seeding_scheme="simple_1", select_green_tokens=True)

generator = Generator(watermark_args, tokenizer, model)
with open("testfile.txt", "r", encoding="utf-8") as f:
    data = f.read()
    
print(data)

# 对数据进行分词
input = tokenizer(data, truncation=False, return_tensors="pt").to(device)

completions_text, completions_tokens  = generator.generate(input_ids=input.input_ids, max_new_tokens=max_gen)

print(completions_tokens)
print(completions_text)

print("####################")


Loading checkpoint shards: 100%|██████████| 8/8 [00:13<00:00,  1.66s/it]


<|User|>:You are a helpful assistant, please answer the following question within 300 words:\n\n( LI5 ) Explain to me the collapse of the Soviet Union . How did one of the largest superpowers manage to fall ? Is it possible something similar could eventually happen to the USA ? Explain like I'm five.<eoh>\n<|Bot|>:


/home/tsq/anaconda3/envs/wmklmm/lib/python3.10/site-packages/transformers/generation/utils.py:1270: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(


[{'text': 'When', 'logprob': -1.390625}, {'text': ' K', 'logprob': -2.40625}, {'text': 'gb', 'logprob': -0.76953125}, {'text': ' officers', 'logprob': -0.126953125}, {'text': ' liquid', 'logprob': -5.0}, {'text': 'ations', 'logprob': -1.1015625}, {'text': ' schools', 'logprob': -4.625}, {'text': ' instead', 'logprob': -3.125}, {'text': ' by', 'logprob': -4.84375}, {'text': ' opening', 'logprob': -1.609375}, {'text': ' their', 'logprob': -0.6953125}, {'text': ' centers', 'logprob': -2.296875}, {'text': ' of', 'logprob': -1.390625}, {'text': ' advanced', 'logprob': -5.375}, {'text': ' ag', 'logprob': -4.1875}, {'text': 'itation', 'logprob': -1.140625}, {'text': ' and', 'logprob': -0.73828125}, {'text': ' infiltr', 'logprob': -1.390625}, {'text': 'ated', 'logprob': -0.69140625}, {'text': ' by', 'logprob': -0.6953125}, {'text': ' humanitarian', 'logprob': -6.8125}, {'text': ' funds', 'logprob': -5.53125}, {'text': ' with', 'logprob': -1.2734375}, {'text': ' NGO', 'logprob': -2.78125}, {'te